# Object Detection using Yolo-RS
The following is the creation of an object detection framework using Yolo-RS.
## Backbone
Firstly, the backbone is created using CBS and RS blocks, the detailed description is provided as follows:
### RS BLocks
https://www.sciencedirect.com/science/article/pii/S105120042400513X
### CBS
https://www.researchgate.net/figure/YOLOv8-network-structure-diagram-CBS-is-composed-of-Convolution-Batch-Normalization_fig2_371983458
### torch.nn
https://medium.com/@sahin.samia/mastering-the-basics-of-torch-nn-a-comprehensive-guide-to-pytorchs-neural-network-module-9f2d704e8c7f
### Python Super()
https://www.geeksforgeeks.org/python-super/
### SPP
https://nanostring.com/products/atomx-spatial-informatics-platform/cosmx-and-atomx-the-first-fully-integrated-single-cell-spatial-solution/?utm_source=bing&utm_medium=cpc&utm_campaign=cosmx&utm_source=bing&utm_medium=cpc&utm_campaign=cosmx&utm_id=SMI_CombinedTier_Search&msclkid=2f0fdb9b7b0611cc8617bc5c805cf0fb

# BackBone

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

### CBS Block

In [ ]:
class CBS (nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride = 1, padding = 0):
        super(CBS, self).__init__()
        if padding is None:
            padding = (kernel_size - 1) // 2
        self.conv = nn.Conv2d (in_channels, out_channels, kernel_size, stride, padding, bias = False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.silu = nn.SiLU()
    def forward(self, x):
        return self.silu(self.bn(self.conv(x)))

### RS Block1

In [ ]:
class RSB1(nn.Module):
    def __init__(self, in_channels):
        super(RSB1, self).__init__()
        c_half = in_channels // 2 #C/2 channels
        self.downsample = CBS(in_channels, in_channels, kernel_size = 3, stride = 2, padding = 1) ## downsampling
        self.br1 = CBS(in_channels, c_half, kernel_size = 1) # Reduced channels
        self.br2 = nn.Sequential (
            CBS(c_half, c_half, kernel_size = 3, padding = 1), #fearture extraction
            CBS(c_half, c_half, kernel_size = 3, padding = 1), #fearture extraction
            CBS(c_half, c_half, kernel_size = 3, padding = 1) #fearture extraction
        )
        self.final_cbs = CBS(in_channels, in_channel, kernel_size = 1) #final transformation
        self.shortcut = CBS(in_channels, in_channels, kernel_size = 3, stride = 2, padding = 1)# identity shortcut
    def forward (self, x):
        identity = self.shortcut(x)
        x = self.downsample(x)
        br1_out = self.br1(x)
        br2_out = self.br2(br1_out)
        x = torch.cat((br1_out, br2_out), dim = 1) #concatinating along channel dimensions
        return self.final_cbs(x) + identity #Residual connection
        

### RS Block2

In [ ]:
class RSB2(nn.Module):
    def __init__(self, in_channels):
        super(RSB2, self).__init__()
        c_half = in_channels // 2
        self.red = CBS (in_channels, c_half, kernel_size = 1)
        self.br1 = nn.Sequential (
            CBS(c_half, c_half, kernel_size = 3, padding = 1), #fearture extraction
            CBS(c_half, c_half, kernel_size = 3, padding = 1) #fearture extraction
        )
        self.br2 = nn. Sequential(
        CBS(c_half, c_half, kernel_size = 3, padding = 1), #fearture extraction
        CBS(c_half, c_half, kernel_size = 3, padding = 1) #fearture extraction
        )
        self.final_cbs = CBS(in_channels, in_channels, kernel_size = 1)
    def forward(self, x):
        identity = x #Direct residual connection
        x = self.red(x)
        br1_out = self.br1(x)
        br2_out = self.br2(x)
        x = torch.cat((br1_out, br2_out), dim = 1)
        return self.final_cbs(x) + identity

### SPP

In [ ]:
class SPP(nn.Module):
    def __init__(self, in_channels):
        super(SPP, self).__init__()
        self.cbs = CBS(in_channels * 4, in_channels, kernel_size = 1)
    def forward (self, x):
        x1 = F.max_pool2d(x, 5, stride = 1, padding = 2)
        x2 = F.max_pool2d(x, 9, stride = 1, padding = 4)
        x3 = F.max_pool2d(x, 13, stride = 1, padding = 6)
        x = torch.cat([x, x1, x2, x3], dim = 1)
        x = self.cbs(x)
        return x

### Implementing Backbone
Now the backbone would be assembled

In [4]:
class BB(nn.Module):
    def __init__(self):
        super(BB, self).__init__()
        self.cbs1 = CBS(3, 32)
        self.RSB1_64 = RSB1(64)
        self.RSB1_128 = RSB1(128)
        self.RSB1_256 = RSB1(256)
        self.RSB1_256 = RSB1(256)
        self.RSB1_512 = RSB1(512)
        self.RSB1_512 = RSB1(512)
        self.RSB1_1024 = RSB1(1024)
        self.RSB1_1024 = RSB1(1024)
        self.spp = SPP(1024)
    def forward(self, x):
        x = self.cbs1(x)
        x = self.RSB1_64(x)
        x = self.RSB1_128(x)
        x = self.RSB1_256(x)
        x = self.RSB1_256(x)
        x = self.RSB1_512(x)
        x = self.RSB1_512(x)
        x = self.RSB1_1024(x)
        x = self.RSB1_1024(x)
        x = self.spp(x)
        return x

NameError: name 'nn' is not defined

# Neck
Now we will move towards the neck module

### Down Module

In [ ]:
##downsamploing block using CBS with stride 2
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Down, self).__init__()
        self.cbs = CBS(in_channels, out_channels, stride = 2)
    def forward(self, x):
        return self.cbs(x)

### Up Module

In [ ]:
##upsample using CBS followed by upsampling layer
class Up(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Up, self).__init__()
        self.cbs = CBS(in_channels, out_channels)
        self.up_s = nn.Upsample(scale_factor = 2, mode = 'nearest')
    def forward(self, x):
        x = self.cbs(x)
        return self.up_s(x)

### Neck

In [5]:
### Neck module, connecting head and backbone
class Neck(nn.Module):
    def __init__(self):
        super(Neck, self).__init__()
        self.ops = nn.ModuleList([
            Down(1024, 512),
            Down(512, 256),
            CBS(256, 128),
            CBS(128, 64),
            Up(64, 128),
            Up(128, 256),
            CBS(256, 512),
            CBS(512, 1024)
        ])
    def forward(self, x):
        for op in self.ops:
            x = op(x)
        return x

IndentationError: expected an indented block (3482084984.py, line 15)

# Yolo Head
Finally, we would create the head part

In [ ]:
class YoloHead(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(YoloHead, self).__init__()
        self.cbs1 = CBS(in_channels, in_channels // 2)
        self.cbs2 = CBS(in_channels // 2, in_channels // 2)
        self.obj = nn.Conv2d(in_channels // 2, 1, kernel_size = 1) #objectness
        self.reg = nn.Conv2d(in_channels // 2, 4, kernel_size = 1) # Bounding box
        self.cls = nn.Conv2d(in_channels // 2, num_classes, kernel_size = 1) # Class score
    def forward(self, x):
        x = self.cbs1(x)
        x = self.cbs2(x)
        obj_out = self.obj(x) #objectness regression
        reg_out = self.reg(x) #bouding box regression
        cls_out = self.cls(x) #class regressison

        return obj_out, reg_out, cls_out

### Main and program running
Finally, we call in main and run the program
### os
https://www.geeksforgeeks.org/os-module-python-examples/
### Transform
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transform.html
### readlines()
https://www.w3schools.com/python/ref_file_readlines.asp
### dataloader
https://pytorch.org/docs/stable/data.html
### criterion
https://nn.readthedocs.io/en/rtd/criterion/index.html
### zero_grad()
https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html
### backward
https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html
### step()
https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html
### device
https://www.geeksforgeeks.org/python-tensorflow-device/
### eval()
https://www.geeksforgeeks.org/eval-in-python/
### ToTensor
https://pytorch.org/vision/master/generated/torchvision.transforms.ToTensor.html
### Transform.Normalize (mean, std)
https://pytorch.org/vision/main/generated/torchvision.transforms.Normalize.html
### torch.cuda
http://pytorch.org/docs/stable/cuda.html
### MSELoss()
https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html
### YoloLoss
https://docs.ultralytics.com/reference/utils/loss/#ultralytics.utils.loss.VarifocalLoss

In [ ]:
# Yolo
class Yolo(nn.Module):
    def __init__(self):
        super(Yolo, self).__init__()
        self.backbone = BB()
        self.neck = Neck()
        self.head = YoloHead (1024, 80)
    def forward(self, x):
        x = self.backbone(x)
        x = self.neck(x)
        x = self.head(x)
        return x

In [ ]:
class Yolodataset(Dataset):
    def __init__(self, img_dir, ann_dir, transform = None):
        self.img_dir = img_dir
        self.ann_dir = ann_dir
        self.transform = transform
        self.img_files = os.listdir (img_dir)
    def __len__(self):
        return len(self.img_files)
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        ann_path = os.path.join(self.ann_dir, self.img_file[idx].replace(".jpg", ".txt"))
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        with open(ann_path, "r") as f:
            anns = f.readlines()
        anns = [list(map(float, ann.strip().split())) for an in ann]
        return img, torch.tensor(anns)

In [ ]:
def train(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for img, targets in dataloader:
        img, targets = img.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(img)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

In [ ]:
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for img, targets in dataloader:
            img, targets = img.to(device), targets.to(device)
            outputs = model(img)
            loss = criterion(outputs, targets)
            running_loss += loss.item()
    return running_loss / len(dataloader)

In [ ]:
def test(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for img, _ in dataloader:
            img = img.to(device)
            outputs = model(img)
            predictions.append(outputs)
    return predictions

In [ ]:
def main():
    train_img_dir = "/"
    train_ann_dir = "/"
    val_img_dir = "/"
    val_ann_dir = "/"
    test_img_dir = "/"
    test_ann_dir = "/"
    #defining transforms
    transform = transforms.Compose([
        transforms.Resize((416, 416)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    ])
    #Datasets creation
    train_dataset = YoloDataset(train_img_dir, train_ann_dir, transform)
    val_dataset = YoloDataset(val_img_dir, val_ann_dir, transform)
    test_dataset = YoloDataset(test_img_dir, test_ann_dir, transform)
    #Loader creations
    train_loader = DataLoader(train_dataset, batch_size = 8, shuffle = True)
    val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = True)
    test_loader = DataLoader(test_dataset, batch_size = 8, shuffle = True)

    #Model Initialization, optimizers and Loss_func
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Yolo().to(device)
    optimizer = optim.Adam(model.parameters(), lr = 0.001)
    criterion = nn.MSELoss() #loss function

    num_epochs = 10
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, optimizer, criterion, device)
        val_loss = validate(model, val_loader, criterion, device)
        print(f"Epoch {epoch+1} / {num_epochs}, Train Loss: {train_loss}, Val Loss: {val_loss}")
    #model predictions
    predictions = test(model, test_loader, device)
    print("Testing")

if __name__ == "__main__":
    main()